# Setup

## Installs

In [1]:
!pip install PyPDF2
!pip install openai
!pip install -U google-generativeai
!pip install flask-ngrok
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


## Imports

In [2]:
import PyPDF2
import google.generativeai as palm
import os
import pandas as pd
import numpy as np
import textwrap
import json

## Document parsing

In [29]:
file_dict = {}

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def process_pdfs_in_folder(folder_path):
    # Check if the folder exists
    if not os.path.isdir(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return

    # Get a list of files in the folder
    files = os.listdir(folder_path)

    # Iterate through each file
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)

        # Check if it's a PDF file
        if file_name.lower().endswith('.pdf'):
            # Extract text from the PDF file
            text = extract_text_from_pdf(file_path)
            file_dict[file_name] = text

process_pdfs_in_folder("/content/Documents")
# Convert the dictionary to a dataframe
df = pd.DataFrame(list(file_dict.items()), columns=['Filename', 'Text'])
df

,Filename,Text
0,System Software.pdf,System Software \nOperating System \nNeed fo...


## API Setup

### API key

In [70]:
API_KEY_GOOGLE_AI='YOUR KEY'

### API Config

In [6]:
palm.configure(api_key=API_KEY_GOOGLE_AI)

## Document Data

In [30]:
df

,Filename,Text
0,System Software.pdf,System Software \nOperating System \nNeed fo...


# Question Generation

## Preparing Prompt

In [61]:
num_questions = 5
total_marks = 10
question_type = "mixed"

content = df.iloc[0]['Text']

def make_prompt(num_questions, question_type, total_marks, content):
    escaped = content.replace("'", "").replace('"', "").replace("\n", " ")
    prompt = textwrap.dedent("""\
        You are a highly intelligent and informative bot that creates questions using text provided from the content provided below
        Please generate '{num_questions}' of type '{question_type}' of total '{total_marks}' marks from available question types that are either fill_in_the_blank, mcqs or descriptive
        Remember the text of these types, only the given text will be used
        if the type is mixed then give a mixture of types of questions
        The result should be a list of JSON objects with these keys in each object ONLY: question_type, question_text, options, marks, marking_scheme
        The marks for each question should be properly weighted. The descriptive questions weigh the most, followed by fill in the blanks and MCQs come in at last
        The total marks of all the questions must be equal to '{total_marks}'
        If type specified above is mixed then all 3 types of questions should be present else only give questions of the type mentioned, the type is '{question_type}'
        Each object must be properly enclosed with curly braces. The content should be able to be parsed using any programming language. Please make sure that any bracket is properly ended
        MCQ options must also contain an "All of the above" and "None" option
        CONTENT: '{content}'
    """).format(num_questions=num_questions, question_type=question_type, total_marks=total_marks, content=content)

    return prompt

prompt = make_prompt(num_questions, total_marks, question_type, content)
print(prompt)

You are a highly intelligent and informative bot that creates questions using text provided from the content provided below
Please generate '5' of type '10' of total 'mixed' marks from available question types that are either fill_in_the_blank, mcqs or descriptive
Remember the text of these types, only the given text will be used
if the type is mixed then give a mixture of types of questions
The result should be a list of JSON objects with these keys in each object ONLY: question_type, question_text, options, marks, marking_scheme
The marks for each question should be properly weighted. The descriptive questions weigh the most, followed by fill in the blanks and MCQs come in at last
The total marks of all the questions must be equal to 'mixed'
If type specified above is mixed then all 3 types of questions should be present else only give questions of the type mentioned, the type is '10'
Each object must be properly enclosed with curly braces. The content should be able to be parsed usi

## Generate Questions

### Pass prompt to Google Gemini

In [62]:
gen_model = palm.GenerativeModel('gemini-pro')
answer = gen_model.generate_content(prompt)

llm_output= answer.text

In [63]:
llm_output

'[\n  {\n    "question_type": "mcq",\n    "question_text": "Which of the following is NOT a key task of memory management in an operating system?",\n    "options": [\n      "Memory protection",\n      "Paging",\n      "Use of virtual memory",\n      "Provision of usernames and passwords",\n      "All of the above",\n      "None"\n    ],\n    "marks": 2,\n    "marking_scheme": "2 marks for correct answer (either None or Provision of usernames and passwords)"\n  },\n  {\n    "question_type": "fill_in_the_blank",\n    "question_text": "The _________________ in an operating system controls access to data sent to and from peripherals.",\n    "options": [],\n    "marks": 1,\n    "marking_scheme": "1 mark for correct answer (Hardware Management)"\n  },\n  {\n    "question_type": "descriptive",\n    "question_text": "Explain the process of fragmentation in file storage and the role of defragmentation software in mitigating its effects.",\n    "options": [],\n    "marks": 5,\n    "marking_schem

### Parse Google Gemini Results

In [66]:
questions = json.loads(llm_output)
questions

[{'question_type': 'mcq',
  'question_text': 'Which of the following is NOT a key task of memory management in an operating system?',
  'options': ['Memory protection',
   'Paging',
   'Use of virtual memory',
   'Provision of usernames and passwords',
   'All of the above',
   'None'],
  'marks': 2,
  'marking_scheme': '2 marks for correct answer (either None or Provision of usernames and passwords)'},
 {'question_type': 'fill_in_the_blank',
  'question_text': 'The _________________ in an operating system controls access to data sent to and from peripherals.',
  'options': [],
  'marks': 1,
  'marking_scheme': '1 mark for correct answer (Hardware Management)'},
 {'question_type': 'descriptive',
  'question_text': 'Explain the process of fragmentation in file storage and the role of defragmentation software in mitigating its effects.',
  'options': [],
  'marks': 5,
  'marking_scheme': '5 marks for a clear and comprehensive explanation of fragmentation and the role of defragmentation s

# Checking Answers

## Prompt Preparation + answer checking results

In [58]:
import re
def check_single_answer(question_number, question, answer, marks, sample_answer):
  prompt = textwrap.dedent("""\
      You are an intelligent system that checks an answer for a question against a marking scheme by comparing semantic similarity
      then you will grade the answer out of the maximum obtainable marks
      and will provide an explanation for why you gave those marks
      The question is,
      QUESTION: {question}
      The provided answer is:
      ANSWER: {answer}
      The sample answer to compare against is
      SAMPLE_ANSWER: {sample_answer}
      The marks out of which to award are {marks}
      Check for meaning of an answer, the idea of the marking scheme and answer must be the same
      Give the results in a stringified JSON object with these keys: question_number: {question_number}, question, answer, obtained_marks, marks: {marks}, explanation
  """).format(question_number=question_number, question=question, answer=answer, marks=marks, sample_answer=sample_answer)

  model = palm.GenerativeModel('gemini-pro')
  answer_results = model.generate_content(prompt)

  llm_output = answer_results.text
  pattern = r'\{.*\}'

  # Search for the JSON object in the response text
  match = re.search(pattern, llm_output, re.DOTALL)

  # If a match is found
  if match:
      # Extract the JSON object from the match
      json_string = match.group(0)
      return json_string
  else:
      return ''

  return ''

# Test Screen

## Setup

### Imports

In [17]:
from pyngrok import ngrok

### NGROK Key

In [69]:
NGROK_AUTH_TOKEN='YOUR_TOKEN'

### NGROK config

In [18]:
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

### User Interface

In [67]:
templates_folder = 'templates'
if not os.path.exists(templates_folder):
    # If not, create the 'templates' folder
    os.makedirs(templates_folder)

html_code_test_screen = """
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <title>{{test_title}}</title>
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <link rel="stylesheet" type="text/css" media="screen" href="main.css" />
    <script src="main.js"></script>
    <link
      href="https://bootswatch.com/5/journal/bootstrap.min.css"
      rel="stylesheet"
    />
    <style>
      /* Custom CSS for alignment */
      .question-container {
        text-align: center;
      }
      .question {
        text-align: left;
      }
      .marks {
        float: right;
      }
    </style>
  </head>
  <body>
    <div class="container question-container">
      <h1>{{test_title}}</h1>
      <form id="test_form">
        {% for question in questions %}
        <div class="card question mb-3">
          <div class="question-card card-body">
            <h5 class="question-text card-title">
              Question {{ loop.index }} ({{question.question_type}})
              {{question.question_text }}
            </h5>
            {% if question.question_type == "fill_in_the_blank" %}
            <input
              type="text"
              class="form-control question-answer"
              id="question{{ loop.index }}"
              name="question{{ loop.index }}"
            />
            {% elif question.question_type == "mcq" %} {% set question_num =
            loop.index %} {% for option in question.options %}
            <div class="form-check">
              <input
                class="form-check-input question-answer"
                type="radio"
                name="question{{ question_num }}"
                id="q{{ question_num }}Option{{ loop.index0 }}"
                value="{{ option }}"
              />
              <label
                class="form-check-label"
                for="q{{ question_num }}Option{{ loop.index0 }}"
              >
                {{ option }}
              </label>
            </div>
            {% endfor %} {% elif question.question_type == "descriptive" %}
            <textarea
              class="form-control question-answer"
              id="question{{ loop.index }}"
              name="question{{ loop.index }} "
              rows="3"
            ></textarea>
            {% endif %}
            <small class="form-text text-muted marks"
              >Marks: {{ question.marks }}</small
            >
          </div>
          <div id="question_{{loop.index}}_marks"></div>
        </div>
        {% endfor %}
        <button id="submit_button" type="submit" class="btn btn-primary">
          Submit
        </button>
        <div class="progress">
          <div
            class="progress-bar progress-bar-striped progress-bar-animated"
            role="progressbar"
            aria-valuenow="100"
            aria-valuemin="0"
            aria-valuemax="100"
            style="width: 75%"
          ></div>
        </div>
      </form>
    </div>
  </body>
  <script
    src="https://code.jquery.com/jquery-3.7.1.min.js"
    integrity="sha256-/JqT3SQfawRcv/BIHPThkBvs0OEvtFFmqPF/lYI/Cxo="
    crossorigin="anonymous"
  ></script>
  <script>
    $(document).ready(function () {
      $("div.progress").hide();
      $("#test_form").submit(function (e) {
        e.preventDefault();

        $("div.progress").show();
        $("#submit_button").hide();

        let answerScript = "";
        let answerScriptJSON = [];
        $(".card.question").each(function (index) {
          let questionText = $(this).find("h5.card-title").text().trim();

          let radioButtons = $(this).find("input[type='radio']:checked");
          let answer;

          if (radioButtons.length > 0) {
            answer = radioButtons.val().trim();
          } else {
            answer = $(this).find("input[type='text'], textarea").val();
          }

          if (answer !== undefined) {
            answer = answer.trim();
          }

          let marks = $(this).find("small.marks").text();
          if (marks !== undefined && marks.trim() !== "") {
            marks = marks.trim().split(":")[1].trim();
          }
          answerScriptJSON.push({
            questionNumber: index + 1,
            question: questionText,
            answer,
            marks,
          });
          answerScript += `QUESTION: ${questionText}\n OBTAINABLE_MARKS: ${marks}\n ANSWER:${answer}\n\n`;
        });

        $.ajax({
          type: "POST",
          url: "{{public_url}}/check-answers",
          contentType: "application/json",
          data: JSON.stringify({
            answer_script: answerScriptJSON,
          }),
          success: function (response) {
            const { graded_answers, message } = response;
            graded_answers.forEach((graded_answer) => {
              const { question_number, explanation, marks, obtained_marks } =
                graded_answer;
              const bgColor =
                obtained_marks === marks
                  ? "bg-success"
                  : obtained_marks < marks && obtained_marks > 0
                  ? "bg-warning"
                  : "bg-danger";
              $(`div#question_${question_number}_marks`).html(`
                <div class="card text-white ${bgColor} mb-3" style="max-width: 18rem;">
                  <div class="card-header">Header</div>
                  <div class="card-body">
                    <h5 class="card-title">${obtained_marks}/${marks}</h5>
                    <p class="card-text">${explanation}</p>
                  </div>
                </div>
              `);
            });
          },
          error: function (xhr, status, error) {
            // Handle the error response
            console.log("Error:", error);
          },
          complete: function (xhr, status) {
            $("div.progress").hide();
            $("#submit_button").show();
          },
        });
      });
    });
  </script>
</html>
"""

# Path to the HTML file
html_file_path = os.path.join(templates_folder, 'test_screen.html')

# Write the HTML code to the file
with open(html_file_path, 'w') as file:
    file.write(html_code_test_screen)

print(f"HTML file 'test_screen.html' has been created in the '{templates_folder}' folder.")

HTML file 'test_screen.html' has been created in the 'templates' folder.


## Web server

### Port

In [20]:
port = 5000

### Helper Functions

In [21]:
def prepare_answer_script(answer_script_json, supplied_questions):
  for i in range(len(supplied_questions)):
    answer_script_json[i]['marking_scheme'] = supplied_questions[i]['marking_scheme']
  return answer_script_json

In [22]:
def grade_answers(answer_script_json):
  graded_answers = []
  for answer in answer_script_json:
    llm_output = check_single_answer(answer['questionNumber'], answer['question'], answer.get('answer', None), answer['marks'], answer['marking_scheme'])
    print(llm_output)
    graded_answers.append(json.loads(llm_output))
  return graded_answers

### Web Server Code

In [68]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)
public_url =  ngrok.connect(port).public_url
test_title = "TestWizard"

@app.route("/")
def home():
    # return f"Running Flask on Google Colab!"
    return render_template("test_screen.html", questions=questions, test_title=test_title, public_url=public_url)

@app.route("/check-answers", methods=["POST"])
def check_answer_script():
    if request.is_json:
          data = request.json

          answer_script = data.get('answer_script')
          answer_script_text = data.get('answer_script_text')
          prepared_answer_script = prepare_answer_script(answer_script, questions)
          print(prepared_answer_script)
          graded_answers = grade_answers(prepared_answer_script)

          return jsonify({
              'message': 'Graded successfully.',
              'graded_answers': graded_answers
          }), 200
    else:
      return jsonify({'error': 'Invalid request. Expected JSON data.'}), 400

print(f"To acces the Gloable link please click {public_url}")

app.run(port=port)

To acces the Gloable link please click https://c9d0-34-138-71-238.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/May/2024 23:06:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/May/2024 23:06:09] "GET /main.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/May/2024 23:06:09] "GET /main.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/May/2024 23:06:10] "GET /favicon.ico HTTP/1.1" 404 -


[{'questionNumber': 1, 'question': 'Question 1 (mcq)\n              Which of the following is NOT a key task of memory management in an operating system?', 'answer': 'Provision of usernames and passwords', 'marks': '2', 'marking_scheme': '2 marks for correct answer (either None or Provision of usernames and passwords)'}, {'questionNumber': 2, 'question': 'Question 2 (fill_in_the_blank)\n              The _________________ in an operating system controls access to data sent to and from peripherals.', 'answer': 'kernel', 'marks': '1', 'marking_scheme': '1 mark for correct answer (Hardware Management)'}, {'questionNumber': 3, 'question': 'Question 3 (descriptive)\n              Explain the process of fragmentation in file storage and the role of defragmentation software in mitigating its effects.', 'answer': 'File fragmentation scatters parts of files across non-contiguous disk sectors, slowing read/write operations. Defragmentation software reorganizes fragmented files into contiguous bl

INFO:werkzeug:127.0.0.1 - - [02/May/2024 23:08:48] "POST /check-answers HTTP/1.1" 200 -


{
  "question_number": 5,
  "question": "______________ saves time and reduces testing time in software development by providing pre-written code.",
  "answer": "DLL",
  "obtained_marks": 1,
  "marks": 1,
  "explanation": "The answer provided is correct as Program Libraries (DLL) are pre-written code that saves time and reduces testing time in software development."
}


...

[{'questionNumber': 1, 'question': 'Question 1 (mcq)\n              Which of the following is NOT a key task of memory management in an operating system?', 'answer': 'Provision of usernames and passwords', 'marks': '2', 'marking_scheme': '2 marks for correct answer (either None or Provision of usernames and passwords)'}, {'questionNumber': 2, 'question': 'Question 2 (fill_in_the_blank)\n              The _________________ in an operating system controls access to data sent to and from peripherals.', 'answer': 'kernel', 'marks': '1', 'marking_scheme': '1 mark for correct answer (Hardware Management)'}, {'questionNumber': 3, 'question': 'Question 3 (descriptive)\n              Explain the process of fragmentation in file storage and the role of defragmentation software in mitigating its effects.', 'answer': 'idk man', 'marks': '5', 'marking_scheme': '5 marks for a clear and comprehensive explanation of fragmentation and the role of defragmentation software'}, {'questionNumber': 4, 'quest

INFO:werkzeug:127.0.0.1 - - [02/May/2024 23:13:55] "POST /check-answers HTTP/1.1" 200 -


{
  "question_number": 5,
  "question": "______________ saves time and reduces testing time in software development by providing pre-written code.",
  "answer": "DLL",
  "obtained_marks": 1,
  "marks": 1,
  "explanation": "The answer provided, \"DLL\", is correct. DLL (Dynamic Link Library) is a type of program library that contains pre-written code that can be used by other programs. This can save time and reduce testing time in software development, as developers do not need to write the code themselves and can instead reuse the code from the library."
}
[{'questionNumber': 1, 'question': 'Question 1 (mcq)\n              Which of the following is NOT a key task of memory management in an operating system?\n            2/2', 'answer': 'Provision of usernames and passwords', 'marks': '2', 'marking_scheme': '2 marks for correct answer (either None or Provision of usernames and passwords)'}, {'questionNumber': 2, 'question': 'Question 2 (fill_in_the_blank)\n              The ____________

INFO:werkzeug:127.0.0.1 - - [02/May/2024 23:14:19] "POST /check-answers HTTP/1.1" 200 -


{
  "question_number": 5,
  "question": "______________ saves time and reduces testing time in software development by providing pre-written code.",
  "answer": "DLL",
  "obtained_marks": 1,
  "marks": 1,
  "explanation": "The answer \"DLL\" is correct because it stands for Dynamic Link Library, which is a type of program library that contains pre-written code that can be used by other programs."
}
